In [1]:
import os
import scipy.io as sio
import numpy as np
import pandas as pd
import time
import ast

from cest_mrf.metrics.dot_product import dot_prod_matching
from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.plot_functions import mask_roi_finder
from my_funcs.plot_functions import mask_check_plot
from my_funcs.plot_functions import compare_txt_method
from my_funcs.plot_functions import t1_t2_pixel_reader
from my_funcs.plot_functions import real_t1_t2
from my_funcs.plot_functions import real_t1_t2_bg
from my_funcs.plot_functions import dict_t1_t2
from my_funcs.plot_functions import dict_fs_ksw
from my_funcs.plot_functions import norm_pixels

from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.cest_functions import dicom_data_arranger
from my_funcs.cest_functions import z_spec_rearranger
from my_funcs.cest_functions import offset_rearranger

from my_funcs.b0_mapping_functions import wassr_b0_mapping
from my_funcs.b0_mapping_functions import b0_correction
from my_funcs.path_functions import make_folder


## 1.1. Enter Data: ##

In [2]:
# # Root stats:
# general_fn = os.path.abspath(os.curdir)
# # Last year
# glu_phantom_fns = [os.path.join(general_fn, 'scans', '23_01_18_glu_phantom_25deg',
#                                 '1_glu_phantom_2p5_5_10mM_ph7_25deg')]  # January 23
# glu_phantom_fns = [os.path.join(general_fn, 'scans', '23_04_04_glu_phantom_37deg',
#                                 '1_glu_phantom_12_16_20mM_ph7_37deg'),
#                    os.path.join(general_fn, 'scans', '23_04_04_glu_phantom_37deg',
#                                 '2_glu_phantom_6_8_4mM_ph7_37deg')]  # April 23
# # My experiments
# glu_phantom_fns = [os.path.join(general_fn, 'scans', '23_12_08_glu_phantom_37deg',
#                                 '1_glu_phantom_12_16_20mM_ph7_37deg'),
#                    os.path.join(general_fn, 'scans', '23_12_08_glu_phantom_37deg',
#                                 '2_glu_phantom_12mM_ph6p5_7_7p4_37deg')]  # December 23
# glu_phantom_fns = [os.path.join(general_fn, 'scans', '24_01_16_glu_phantom_37deg',
#                                 '1_glu_phantom_10_20_30mM_ph7_37deg'),
#                    os.path.join(general_fn, 'scans', '24_01_16_glu_phantom_37deg',
#                                 '2_glu_phantom_5_10_15mM_ph7_37deg'),
#                    os.path.join(general_fn, 'scans', '24_01_16_glu_phantom_37deg',
#                                 '3_glu_phantom_20mM_ph6p8_7_7p2_37deg')]  # January 24
# glu_phantom_fns = [os.path.join(general_fn, 'scans', '24_02_12_glu_phantom_vardeg',
#                                 '1_glu_phantom_20_16_12mM_ph7_dec_33deg'),
#                    os.path.join(general_fn, 'scans', '24_02_12_glu_phantom_vardeg',
#                                 '2_glu_phantom_20_16_12mM_ph7_dec_37deg'),
#                    os.path.join(general_fn, 'scans', '24_02_12_glu_phantom_vardeg',
#                                 '3_glu_phantom_20_16_12mM_ph7_dec_40deg'),
#                    os.path.join(general_fn, 'scans', '24_02_12_glu_phantom_vardeg',
#                                 '4_glu_phantom_20_16_12mM_ph7_feb_33deg'),
#                    os.path.join(general_fn, 'scans', '24_02_12_glu_phantom_vardeg',
#                                 '5_glu_phantom_20_16_12mM_ph7_feb_37deg')]  # February 24


In [3]:
# Root stats:
general_fn = os.path.abspath(os.curdir)
# Subject stats:
glu_phantom_fns = [os.path.join(general_fn, 'scans', 'data_glutamate_phantoms_jan16_2024',
                                '20240116_114710_OrPerlman_phantom_glu_5_10_15mM_pH7_a_1_1')]  # January

txt_file_name = 'labarchive_notes.txt'

# dict range
f_const = 3 / 110000
ranges = [
    {'fs_0': (0, 30*f_const),
    'ksw_0': (3000, 7000),
     't1w': (3.5, 4.6),
     't2w': (1.45, 1.8)}
    ]


In [4]:
conc_ls = [['12 mM', '16 mM', '20 mM'],
           ['16 mM', '20 mM', '12 mM'],
           ['16 mM', '12 mM', '20 mM'],
           ['12 mM', '20 mM', '16 mM'],
           ['12 mM', '20 mM', '16 mM'],
           ['20 mM', '12 mM', '16 mM']]

ph_ls = [['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7', 'pH 7']]

tags = [['c', 'b', 'a'], 
        ['b', 'a', 'c'],
        ['b', 'c', 'a'],
        ['c', 'a', 'b'],
        ['c', 'a', 'b'],
        ['a', 'c', 'b']]

tag_x_locs = [[-3,3,-2],
              [-2,1,-2],
              [7, 0, 15],
              [0,-3,0],
              [0,-3,0],
              [0,-3,0]]

tag_y_locs = [[-13,13,14],
              [-13,-13,12],
              [-13,-13,12],
              [-13,12,12],
              [-13,12,12],
              [-13,13,12]]

temperatures = ['33°C', '37°C', '40°C', '33°C', '37°C', '37°C']
months = ['dec', 'dec', 'dec', 'feb', 'feb', 'dec_og']

# vis_params = {
#     'month': 'Dec',
#     'temp': '33°C',
#     'conc_l': ['12 mM', '16 mM', '20 mM'],
#     'ph_ls': ['pH 7', 'pH 7', 'pH 7'],
#     'tag_x_locs': [-3,3,-2],
#     'tag_y_locs': [-13,13,14]
# }
    

# run all dicts #

In [5]:
fp_prtcl_names = ['107a', '107d']  # to be looped later with all options

# for phantom_i in range(len(glu_phantom_fns)):
for phantom_i in range(1):
    phantom_i = 4
    print(f'################################# start of phantom {phantom_i+1} #################################')
    glu_phantom_fn = glu_phantom_fns[phantom_i]
    val_range = ranges[phantom_i]
    txt_file_name = 'labarchive_notes.txt'

    for fp_prtcl_name in fp_prtcl_names:
        glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, fp_prtcl_name)
        glu_acquired_data = dicom_data_arranger(bruker_dataset, glu_phantom_dicom_fn)

        # create acquired data folder: root->scans->date->subject->E->mrf_files->acquired_data
        acquired_data_fn = os.path.join(glu_phantom_mrf_files_fn, 'acquired_data.mat')
        make_folder(glu_phantom_mrf_files_fn)

        # # save acquired data to: root->scans->date->subject->E->mrf_files->acquired_data
        # sio.savemat(acquired_data_fn, {'acquired_data': glu_acquired_data})
        # print(f'Acquired data was saved as {glu_acquired_data.shape} sized array')

        start = time.perf_counter()
        dict_fn = os.path.join('exp', 'feb', fp_prtcl_name, 'dict.csv')
        synt_df = pd.read_csv(dict_fn, header=0)

        # filter dict as needed
        dict_fn = os.path.join('exp', 'feb', fp_prtcl_name, 'f_dict.csv')
        df_masks = [synt_df[column].between(min_val, max_val) for column, (min_val, max_val) in val_range.items()]
        filtered_df = synt_df[np.all(df_masks, axis=0)]
        filtered_df.to_csv(path_or_buf=dict_fn, index=False)  # To save as CSV without row indices

        quant_maps = dot_prod_matching(dict_fn=dict_fn, acquired_data_fn=acquired_data_fn)  # I changed it!!!
        end = time.perf_counter()
        s = (end - start)
        print(f"Dot product matching took {s:.03f} s.")

        # save acquired data to: root->scans->date->subject->E->mrf_files->quant_maps.mat
        quant_maps_fn = os.path.join(glu_phantom_mrf_files_fn, 'quant_maps.mat')
        sio.savemat(quant_maps_fn, quant_maps)
        print('quant_maps.mat saved')

    print(f'################################# end of phantom {phantom_i+1} #################################')


################################# start of phantom 5 #################################


IndexError: list index out of range

# Choose phantom to visualize #

In [ ]:
phantom_choice = 5  # choose 1-6
fp_prtcl_name = '107a'
glu_phantom_fn = glu_phantom_fns[phantom_choice-1]
txt_file_name = 'labarchive_notes.txt'
val_range = ranges[phantom_choice-1]
conc_l = conc_ls[phantom_choice-1]
ph_l = ph_ls[phantom_choice-1]
tag = tags[phantom_choice-1]
tag_x_loc = tag_x_locs[phantom_choice-1]
tag_y_loc = tag_y_locs[phantom_choice-1]
temp = temperatures[phantom_choice-1]
month = months[phantom_choice-1]


## 1.2. Create bruker dataset and masks ##

In [ ]:
glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, fp_prtcl_name)

_, _, bruker_dataset_mask = bruker_dataset_creator(glu_phantom_fn, txt_file_name, '107a')  # always takes mask from 107a
vial_rois, full_mask, bg_mask = mask_roi_finder(bruker_dataset_mask)

quant_data_fn = os.path.join(glu_phantom_mrf_files_fn, 'quant_maps.mat')
quant_maps = sio.loadmat(quant_data_fn)
fs_array = quant_maps['fs'] * 110e3 / 3
ksw_array = quant_maps['ksw']
dp_array = quant_maps['dp']
t1w_array = quant_maps['t1w'] * 1000
t2w_array = quant_maps['t2w'] * 1000


## 2.1. plot 1- mask check ##

In [ ]:
mask_check_plot(vial_rois, full_mask, bg_mask, conc_l, ph_l)


## 2.2. plot 2- parameter check ##

In [ ]:
prtcl_txt_fn = os.path.join(general_fn, 'protocol_text_files', 'glutamate', f'{fp_prtcl_name}.txt')

compare_txt_method(prtcl_txt_fn, bruker_dataset, fp_prtcl_name)


## 2.3. t1 & t2 check ##

In [ ]:
t1_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_phantom_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t1')
t2_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_phantom_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t2')

real_t1_t2(t1_pixels, t2_pixels, full_mask, vial_rois, tag, tag_x_loc, tag_y_loc, phantom_choice, temp, month, conc_l, ph_l)
real_t1_t2_bg(t1_pixels, t2_pixels, bg_mask, phantom_choice, temp, month)
dict_t1_t2(t1w_array, t2w_array, full_mask, vial_rois, tag, tag_x_loc, tag_y_loc, fp_prtcl_name, phantom_choice, temp, month, conc_l, ph_l)


## 3. fs & ksw maps ##

In [ ]:
fs_lim = [f / f_const for f in list(val_range['fs_0'])] 
kws_lim = list(val_range['ksw_0'])
dict_fs_ksw(fs_array, ksw_array, full_mask, vial_rois, tag, tag_x_loc, tag_y_loc, fp_prtcl_name, phantom_choice, temp, month, conc_l, ph_l, [fs_lim, kws_lim])


## 4. Signal close look ##

In [ ]:
start = time.perf_counter()
dict_fn = os.path.join('exp', 'feb', fp_prtcl_name, 'dict.csv')
synt_df = pd.read_csv(dict_fn, header=0)

# filter dict as needed
dict_fn = os.path.join('exp', 'feb', fp_prtcl_name, 'f_dict.csv')
df_masks = [synt_df[column].between(min_val, max_val) for column, (min_val, max_val) in val_range.items()]
filtered_df = synt_df[np.all(df_masks, axis=0)]
filtered_df.to_csv(path_or_buf=dict_fn, index=False)  # To save as CSV without row indices


In [ ]:
from plotly.subplots import make_subplots
from my_funcs.plot_functions import norm_roi
import plotly.graph_objects as go
from my_funcs.plot_functions import vial_locator
import plotly.io as pio

abc_id = [tag.index('a'), tag.index('b'), tag.index('c')]  # [2, 0, 1]
abc_int = np.argsort(np.argsort(tag))  # [1, 2, 0]
fig = make_subplots(rows=3, cols=1, vertical_spacing=0.07, horizontal_spacing=0.01,
                    subplot_titles=[f'(a) {conc_l[abc_id[0]]}, {ph_l[abc_id[0]]}',
                                        f'(b) {conc_l[abc_id[1]]}, {ph_l[abc_id[1]]}',
                                        f'(c) {conc_l[abc_id[2]]}, {ph_l[abc_id[2]]}'])

# Colors for the scatter plots
color_1 = '#1f77b4'
color_2 = '#ff7f0e'
color_3 = '#2ca02c'

vial_rois, full_mask, bg_mask = mask_roi_finder(bruker_dataset_mask, 3)
roi_masks, vial_loc = vial_locator(full_mask, vial_rois)

for roi_i, roi_mask in enumerate(roi_masks):
    mean_sigs, std_sigs, _ = norm_roi(roi_mask, quant_data_fn, dict_fn, glu_phantom_mrf_files_fn)  # real, matched, expected
    
    x_axis = np.arange(0, len(mean_sigs[0]))
    # # Add heatmaps for the three arrays
    # scatter_real = go.Scatter(x=x_axis, y=mean_sigs[0], mode='lines', name='Real Signal', line=dict(color=color_1),
    #                       error_y=dict(type='data', # value of error bar given in data coordinates
    #                                    array=std_sigs[0],
    #                                     visible=True))
    # scatter_match = go.Scatter(x=x_axis, y=mean_sigs[1], mode='lines', name='Matched Signal', line=dict(color=color_2),
    #                           error_y=dict(type='data', # value of error bar given in data coordinates
    #                                        array=std_sigs[1],
    #                                        visible=True))
    # Add heatmaps for the three arrays
    scatter_real = go.Scatter(x=x_axis, y=mean_sigs[0], mode='lines', name='Real Signal', line=dict(color=color_1))
    scatter_match = go.Scatter(x=x_axis, y=mean_sigs[1], mode='lines', name='Matched Signal', line=dict(color=color_2))
    
    # Add traces to the first subplot
    fig.add_trace(scatter_real, row=abc_int[roi_i] + 1, col=1)
    fig.add_trace(scatter_match, row=abc_int[roi_i] + 1, col=1)
    
    fig.update_yaxes(title_text=f'Normalized signal', row=roi_i+1, col=1)

# Set layout for better visualization
fig.update_layout(
    template='plotly_white',  # Set the theme to plotly white
    title_text=f'Phantom {month} {temp} - {fp_prtcl_name}, signal close look',
    showlegend=True,  # Hide legend
    legend=dict(x=0.8, y=1.05),
    height=600,
    width=500  # Set a width based on your preference
)

labels_to_show_in_legend = ['Real Signal', 'Matched Signal']

# only show 3 first traces in legend!
for trace_i, trace in enumerate(fig['data']):
    if trace_i > 1:
        trace['showlegend'] = False

# Adjust margin to reduce whitespace
fig.update_layout(margin=dict(l=0, r=5, t=60, b=0))
# Set y-axis ticks to values in the range [0, 1] with a step of 0.2
fig.update_yaxes(tickmode='linear', tick0=0, dtick=0.1, range=[0.12, 0.25])

# Show the plot
fig.show()
pio.write_image(fig, f'images/phantom_{phantom_choice}//{fp_prtcl_name}_sigs.jpeg')


## 5. T1 T2 plots (December phantom) ##

In [ ]:
# n = 3
# t1_20_means, t1_20_stds = np.zeros([n]), np.zeros([n])
# t1_16_means, t1_16_stds = np.zeros([n]), np.zeros([n])
# t1_12_means, t1_12_stds = np.zeros([n]), np.zeros([n])
# t1_bg_means, t1_bg_stds = np.zeros([n]), np.zeros([n])
# t2_20_means, t2_20_stds = np.zeros([n]), np.zeros([n])
# t2_16_means, t2_16_stds = np.zeros([n]), np.zeros([n])
# t2_12_means, t2_12_stds = np.zeros([n]), np.zeros([n])
# t2_bg_means, t2_bg_stds = np.zeros([n]), np.zeros([n])
# 
# for i in range(n):
#     phantom_i = i
#     conc_l = conc_ls[i]
#     ph_l = ph_ls[i]
#     tag = tags[i]
#     tag_x_loc = tag_x_locs[i]
#     tag_y_loc = tag_y_locs[i]
#     glu_phantom_fn = glu_phantom_fns[i]
# 
#     glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, fp_prtcl_name)
#     _, _, bruker_dataset_mask = bruker_dataset_creator(glu_phantom_fn, txt_file_name, '107a')  # always takes mask from 107a
#     vial_rois, full_mask, bg_mask = mask_roi_finder(bruker_dataset_mask, 3)
#     t1_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_phantom_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t1')
#     t2_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_phantom_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t2')
# 
#     t1_t2_df = real_t1_t2(t1_pixels, t2_pixels, full_mask, vial_rois, tag, tag_x_loc, tag_y_loc, phantom_choice, conc_l, ph_l)
# 
#     _, _, bg_mask = mask_roi_finder(bruker_dataset_mask)
#     t1_t2_bg_df = real_t1_t2_bg(t1_pixels, t2_pixels, bg_mask, phantom_choice)
#     t1_20_means[i] = t1_t2_df.iloc[0,2]
#     t1_20_stds[i] = t1_t2_df.iloc[0,4]
#     t1_16_means[i] = t1_t2_df.iloc[1,2]
#     t1_16_stds[i] = t1_t2_df.iloc[1,4]
#     t1_12_means[i] = t1_t2_df.iloc[2,2]
#     t1_12_stds[i] = t1_t2_df.iloc[2,4]
#     t1_bg_means[i] = t1_t2_bg_df.iloc[0,2]
#     t1_bg_stds[i] = t1_t2_bg_df.iloc[0,4]
# 
#     t2_20_means[i] = t1_t2_df.iloc[1,3]
#     t2_20_stds[i] = t1_t2_df.iloc[1,5]
#     t2_16_means[i] = t1_t2_df.iloc[1,3]
#     t2_16_stds[i] = t1_t2_df.iloc[1,5]
#     t2_12_means[i] = t1_t2_df.iloc[2,3]
#     t2_12_stds[i] = t1_t2_df.iloc[2,5]
#     t2_bg_means[i] = t1_t2_bg_df.iloc[0,3]
#     t2_bg_stds[i] = t1_t2_bg_df.iloc[0,5]


In [ ]:
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import plotly.io as pio
# 
# # Sample data for means and stds (replace this with your actual data)
# means_t1 = [t1_20_means, t1_16_means, t1_12_means, t1_bg_means]
# stds_t1 = [t1_20_stds, t1_16_stds, t1_12_stds, t1_bg_stds]
# means_t2 = [t2_20_means, t2_16_means, t2_12_means, t2_bg_means]
# stds_t2 = [t2_20_stds, t2_16_stds, t2_12_stds, t2_bg_stds]
# 
# # Temperature labels
# temperatures = ['33°C', '37°C', '40°C']
# 
# # Create scatter trace with error bars
# def create_trace(means, stds, row_idx, col_idx):
#     trace = go.Scatter(
#         x=temperatures,
#         y=means,
#         mode='markers+lines',
#         name=f'Row {row_idx}, Col {col_idx}',
#         error_y=dict(
#             type='data',
#             array=stds,
#             visible=True
#         )
#     )
#     return trace
# 
# # Create subplots
# fig = make_subplots(rows=2, cols=4, subplot_titles=['T1 20 mM', 'T1 16 mM', 'T1 12 mM', 'T1 Background',
#                                                     'T2 20 mM', 'T2 16 mM', 'T2 12 mM', 'T2 Background'],
#                     horizontal_spacing=0.05, vertical_spacing=0.15)
# 
# # Populate subplots with traces
# for i in range(4):
#     # T1
#     fig.add_trace(create_trace(means_t1[i], stds_t1[i], 1, i+1), row=1, col=i+1)
#     # T2
#     fig.add_trace(create_trace(means_t2[i], stds_t2[i], 2, i+1), row=2, col=i+1)
# 
# # Update layout for each subplot
# for i in range(2):
#     fig.update_yaxes(title_text='Mean [ms]', row=i+1, col=1)
# 
# # Update layout for each subplot
# fig.update_yaxes(range=[3500, 4700], row=1)
# fig.update_yaxes(range=[0, 2500], row=2)
# fig.update_yaxes(range=[400, 1000], row=2)
# fig.update_yaxes(range=[0, 2500], row=2, col=4)
# 
# # Update layout
# fig.update_layout(
#     template='plotly_white',
#     title='T1 and T2 for Different Temperatures and Concentrations',
#     height=600,
#     width=1200,
#     showlegend=False  # Show legend for each subplot
# )
# 
# # Show plot
# fig.show()
# # pio.write_image(fig, f'images/t1_t2_scatter_dec.jpeg')
